In [8]:
import mysql.connector
import cursor
import pandas as pd
from sqlalchemy import create_engine        #para cargar las tablas de mysql

In [9]:
df_alquileres = pd.read_csv(r'C:\Users\franp\Escritorio\Bootcamp\Proyecto Análisis Mercado Inmobiliario\Sprint 2\08032025\alquileres_completo_limpio.csv')
df_compras = pd.read_csv(r'C:\Users\franp\Escritorio\Bootcamp\Proyecto Análisis Mercado Inmobiliario\Sprint 2\08032025\compras_completo_limpio.csv')
df_alquileres_clustering = pd.read_csv(r'C:\Users\franp\Escritorio\Bootcamp\Proyecto Análisis Mercado Inmobiliario\Sprint 2\df_alquileres_clustering.csv')
df_compras_clustering = pd.read_csv(r'C:\Users\franp\Escritorio\Bootcamp\Proyecto Análisis Mercado Inmobiliario\Sprint 2\df_compras_clustering.csv')

C:\Users\franp\AppData\Local\Temp\ipykernel_17400\1404999507.py:2: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_compras = pd.read_csv(r'C:\Users\franp\Escritorio\Bootcamp\Proyecto Análisis Mercado Inmobiliario\Sprint 2\08032025\compras_completo_limpio.csv')


In [ ]:
#quito el mapeo de nuevo orden de variables y de mapa_ccaa porque ya está incluido en el csv
#quitamos también el mapeo de ccaa

## NO EJECUTAR - Generar el esquema SQL - Mapeo de tipo de datos en MySQL - OPCIÓN 1

In [10]:
#este código es más extensible pero a la vez flexible por si se quieren añadir más tipo de variables

In [10]:
#mapeo de tipos de datos de pandas a MySQL

dtype_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',
    'bool': 'BOOLEAN',
    'datetime64': 'DATETIME'
}

#función para generar la estructura SQL

def generate_mysql_schema(df, table_name):
    sql = f'CREATE TABLE {table_name} (\n'

    for col, dtype in df.dtypes.items():             #dtypes: cada tipo de dato de las variables
        mysql_type = dtype_mapping.get(str(dtype), 'TEXT')
        sql += f"    `{col}` {mysql_type},\n"

    sql = sql.rstrip(',\n') + '\n);'
    return sql

#generar el código SQL (este es el que se introduce en sql_schema)
sql_alquileres = generate_mysql_schema(df_alquileres, 'general_alquileres')
sql_compras = generate_mysql_schema(df_compras, 'general_compras')

#los guardamos en un archivo mysql porque no se puede leer completo (truncado)
with open('general_alquileres_mysql.sql', 'w') as f:
    f.write(sql_alquileres)

with open('general_compras_mysql.sql', 'w') as f:
    f.write(sql_compras)

print('Esquemas generados y guardados exitosamente')


Esquemas generados y guardados exitosamente


In [12]:
print(f'Columnas en alquileres: {df_alquileres.columns.tolist()}')
print(f'Columnas en compras: {df_compras.columns.tolist()}')

Columnas en alquileres: ['identificador', 'nombre', 'href', 'agencia', 'area', 'provincia', 'comunidad_autonoma', 'coordenadas', 'tipo_de_casa', 'planta', 'orientacion', 'precio', 'precio_m2', 'gastos_de_comunidad', 'antiguedad', 'conservacion', 'telefono', 'actualizacion', 'timestamp', 'superficie', 'superficie_util', 'superficie_construida', 'superficie_solar', 'habitaciones', 'baños', 'comedor', 'cocina_equipada', 'amueblado', 'lavadero', 'balcon', 'terraza', 'trastero', 'garaje', 'piscina', 'chimenea', 'jardin', 'exterior', 'carpinteria_interior', 'carpinteria_exterior', 'tipo_suelo', 'puerta_blindada', 'armarios_empotrados', 'luz', 'agua', 'gas', 'calefaccion', 'aire_acondicionado', 'sistema_de_seguridad', 'ascensor', 'portero_automatico', 'se_aceptan_mascotas', 'zona_comunitaria', 'sala_comunitaria', 'adaptado_a_personas_con_movilidad_reducida', 'zona_de_juegos_infantiles', 'cuarto_de_bicicletas', 'calle_alumbrada', 'alcantarillado', 'instalaciones_deportivas', 'certificado_energ

## Generar el esquema SQL automáticamente - OPCIÓN 2


In [ ]:
#este código es más conciso y útil para mapear tipos básicos

In [11]:
def inferir_tipo_sql(dtype):
    """inferir el tipo de dato SQL a partir del tipo de dato en Pandas."""

    if pd.api.types.is_int64_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    else:
        return 'VARCHAR(255)'

def generar_schema(tabla, df, primary_key=None, foreign_key=None, referenced_table=None):
    """generar la sentencia SQL para la creación de una tabla con clave primaria y foránea."""
    
    columnas = []
    for col, dtype in zip(df.columns, df.dtypes):
        tipo_sql = inferir_tipo_sql(dtype)
        columnas.append(f'`{col}` {tipo_sql}')
    if primary_key is None:
        columnas.append('`id` INT AUTO_INCREMENT PRIMARY KEY')
    else:
        columnas.append(f'PRIMARY KEY (`{primary_key}`)')

    if foreign_key and referenced_table:
        columnas.append(f'FOREIGN KEY (`{foreign_key}`) REFERENCES `{referenced_table}`(`id`) ON DELETE CASCADE')
    
    return f'CREATE TABLE {tabla} (\n    '+',\n    '.join(columnas) + '\n);'

sql_schema_alquileres = generar_schema('general_alquileres', df_alquileres)
sql_schema_compras = generar_schema('general_compras', df_compras)

sql_schema_cluster_alquileres = generar_schema(
    'general_alquileres_cluster', df_alquileres_clustering,
    primary_key='id', foreign_key='id', referenced_table='general_alquileres'
    )

sql_schema_cluster_compras = generar_schema(
    'general_compras_cluster', df_compras_clustering,
    primary_key='id', foreign_key='id', referenced_table='general_compras'
    )

print(sql_schema_alquileres)
print(sql_schema_compras)
print(sql_schema_cluster_alquileres)
print(sql_schema_cluster_compras)


CREATE TABLE general_alquileres (
    `identificador` VARCHAR(255),
    `nombre` VARCHAR(255),
    `href` VARCHAR(255),
    `agencia` VARCHAR(255),
    `area` VARCHAR(255),
    `provincia` VARCHAR(255),
    `comunidad_autonoma` VARCHAR(255),
    `coordenadas` VARCHAR(255),
    `tipo_de_casa` VARCHAR(255),
    `planta` INT,
    `orientacion` VARCHAR(255),
    `precio` FLOAT,
    `precio_m2` FLOAT,
    `gastos_de_comunidad` VARCHAR(255),
    `antiguedad` VARCHAR(255),
    `conservacion` VARCHAR(255),
    `telefono` VARCHAR(255),
    `actualizacion` VARCHAR(255),
    `timestamp` VARCHAR(255),
    `superficie` FLOAT,
    `superficie_util` FLOAT,
    `superficie_construida` FLOAT,
    `superficie_solar` FLOAT,
    `habitaciones` FLOAT,
    `baños` FLOAT,
    `comedor` VARCHAR(255),
    `cocina_equipada` VARCHAR(255),
    `amueblado` VARCHAR(255),
    `lavadero` VARCHAR(255),
    `balcon` VARCHAR(255),
    `terraza` VARCHAR(255),
    `trastero` VARCHAR(255),
    `garaje` VARCHAR(255),
    `p

C:\Users\franp\AppData\Local\Temp\ipykernel_17400\4038514233.py:4: DeprecationWarning: is_int64_dtype is deprecated and will be removed in a future version. Use dtype == np.int64 instead.
  if pd.api.types.is_int64_dtype(dtype):
C:\Users\franp\AppData\Local\Temp\ipykernel_17400\4038514233.py:4: DeprecationWarning: is_int64_dtype is deprecated and will be removed in a future version. Use dtype == np.int64 instead.
  if pd.api.types.is_int64_dtype(dtype):
C:\Users\franp\AppData\Local\Temp\ipykernel_17400\4038514233.py:4: DeprecationWarning: is_int64_dtype is deprecated and will be removed in a future version. Use dtype == np.int64 instead.
  if pd.api.types.is_int64_dtype(dtype):
C:\Users\franp\AppData\Local\Temp\ipykernel_17400\4038514233.py:4: DeprecationWarning: is_int64_dtype is deprecated and will be removed in a future version. Use dtype == np.int64 instead.
  if pd.api.types.is_int64_dtype(dtype):


## Conexión a la base de datos

In [12]:
#parámetros de conexión
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Amsterdam1991!',
    database='analisis_inmobiliario'
)

cursor = connection.cursor()

try:
    cursor.execute(sql_schema_alquileres)
    print("Tabla general_alquileres creada exitosamente.")

    cursor.execute(sql_schema_compras)
    print('Tabla general_compras creada exitosamente')

    cursor.execute(sql_schema_cluster_alquileres)
    print("Tabla general_alquileres_cluster creada exitosamente.")

    cursor.execute(sql_schema_cluster_compras)
    print("Tabla general_compras_cluster creada exitosamente.")


    connection.commit()

except mysql.connector.Error as err:
    print(f'Error: {err}')

cursor.close()
connection.close()

Error: 1050 (42S01): Table 'general_alquileres' already exists


## Insertar datos desde el CSV a las tablas

In [ ]:
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Amsterdam1991!',
    database='analisis_inmobiliario'
)

cursor = connection.cursor()

def insertar_datos(tabla, df):
    columnas = ', '.join([f"`{col}`" for col in df.columns])
    valores = ', '.join(['%s'] * len(df.columns))
    query = f"INSERT INTO {tabla} ({columnas}) VALUES ({valores})"

    for _, row in df.iterrows():
        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            print(f"Error al insertar fila: {e}")
            print(row)

    connection.commit()
    print(f"{len(df)} registros insertados en '{tabla}'.")    

#insertar datos de los csv en las tablas
insertar_datos("general_alquileres", df_alquileres)
insertar_datos("general_compras", df_compras)
insertar_datos("general_alquileres_cluster", df_alquileres)
insertar_datos("general_compras_cluster", df_compras)

#cerrar la conexión
cursor.close()
connection.close()
print('Conexión cerrada')

13483 registros insertados en 'general_alquileres'.
